In [ ]:
import tensorflow as tf
import numpy as np

print("\n========== 연산 ==========\n")

print(tf.math.add(1, 2))                      # 합
print(tf.math.add([1, 2], [3, 4]))            # 합
print(tf.math.square(5))                      # 제곱
print(tf.math.reduce_sum([1, 2, 3]))          # 전체
print(tf.math.square(2) + tf.math.square(3))  # 제곱과 제곱의 합

print("\n========== 크기와 데이터 타입 ==========\n")

x = tf.linalg.matmul([[1]], [[2, 3]])   # 행렬 곱셈
print(x)
print(x.shape)
print(x.dtype)

print("\n========== Tensor와 NumPy의 호환성 ==========\n")

ndarray = np.ones([3, 3])
tensor = tf.math.multiply(ndarray, 42)

print("TensorFlow operations convert numpy arrays to Tensors automatically")
print(tensor)
print("And NumPy operations convert Tensors to NumPy arrays automatically")
print(np.add(tensor, 1))
print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())





========== 연산 ==========

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)

========== 크기와 데이터 타입 ==========

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>

========== Tensor와 NumPy의 호환성 ==========

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to NumPy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [1]:
import tensorflow as tf

# 필요 시 텐서를 CPU와 GPU 메모리 사이에서 복사
# 연산에 의해 생성된 텐서는 전형적으로 연산이 실행된 장치의 메모리에 의해 실행

print("\n========== GPU 가속 ==========\n") # 런타임을 GPU로 바꾸고 실행하기

x = tf.random.uniform([3, 3])

print("Is there a GPU available :", tf.config.list_physical_devices("GPU"))
print("Is the Tensor on GPU #0 :", x.device.endswith('GPU:0'))



========== GPU 가속 ==========

Is there a GPU available : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is the Tensor on GPU #0 : True


In [2]:
import tensorflow as tf
import time

# 명시적인 지침이 제공되지 않으면 TensorFlow는 연산을 실행할 장치를 자동으로 결정하고 필요한 경우 해당 장치에 텐서를 복사
# TensorFlow 연산은 tf.device 컨텍스트 관리자를 사용하여 특정 장치에 명시적으로 배치

print("\n========== 명시적 장치 배치 ==========\n")

def time_matmul(x):
  start = time.time()                         # 시작 시간을 기록
  for loop in range(10):                      # 10번 반복하여 실행
    tf.linalg.matmul(x, x)                    # 행렬 곱셈 연산
  result = time.time() - start                # 끝난 후 시간을 측정
  print(f"10 loops: {(1000*result):0.2f}ms")  # 밀리초 단위로 실행 시간 출력

print("On CPU:")
with tf.device("CPU:0"):                      # CPU에서 실행
  x = tf.random.uniform([1000, 1000])         # 1000x1000 크기의 무작위 텐서를 생성
  assert x.device.endswith("CPU:0")           # x 텐서가 CPU에 배치되었는지 확인
  time_matmul(x)                              # 행렬 곱셈 연산 시간을 측정

print("On CPU:")
with tf.device("CPU:0"):                      # CPU에서 실행
  x = tf.random.uniform([1000, 1000])         # 1000x1000 크기의 무작위 텐서를 생성
  assert x.device.endswith("CPU:0")           # x 텐서가 CPU에 배치되었는지 확인
  time_matmul(x)                              # 행렬 곱셈 연산 시간을 측정



========== 명시적 장치 배치 ==========

On CPU:
10 loops: 287.85ms
On GPU:
10 loops: 111.09ms


In [8]:
# 데이터세트

ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6]) # 결과적으로 ds_tensors는 [1], [2], [3], [4], [5], [6]과 같은 6개의 요소를 갖는 데이터셋이 된다.

# CSV 파일 생성
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("Line1\nLine2\nLine3")

ds_file = tf.data.TextLineDataset(filename) # 지정된 파일을 읽고 각 줄을 텍스트 항목으로 나누어 데이터셋을 생성

# 변환 적용

ds_tensor = ds_tensors.map(tf.math.square).shuffle(2).batch(2)  # map 함수는 데이터셋의 각 요소에 주어진 함수를 적용한다. tf.math.square는 입력값의 제곱을 계산한다. shuffle(N)은 버퍼 크기를 N으로 지정해, 데이터셋의 요소가 N개씩 임시로 저장되어 섞인 후, 다시 원래 데이터셋에 삽입한다. 데이터셋을 지정된 크기의 배치로 묶는다.
ds_file = ds_file.batch(2)                                      # batch를 2로 적용해 Line1과 Line2가 첫번째 배치로 Line3rk 두번째 배치로 묶은다.

# 반복

print('\nElements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)



Elements of ds_tensors:
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line1' b'Line2'], shape=(2,), dtype=string)
tf.Tensor([b'Line3'], shape=(1,), dtype=string)
